In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from adan_pytorch import Adan
import copy
# from utils import preparte_data_loader
# from utils import set_parameter_requires_grad
from utils import DEVICE, LOSS_CRITERIA
import time
from utils import DEVICE, AudioDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from transformers import HubertConfig, HubertForSequenceClassification

/home/mlspeech/royweber/anaconda3/envs/roy_lang/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### double check our model to ensure the only train samples are for the last part

In [2]:
# def train_model(model, dataloaders, optimizer, num_epochs=25, device=DEVICE, loss_criteria = LOSS_CRITERIA):
#     since = time.time()

#     val_acc_history = []
#     training_loss = []
    

#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # forward
#                 # track history if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     # Get model outputs and calculate loss
#                     outputs = model(inputs)
#                     loss = loss_criteria(outputs, labels)

#                     _, preds = torch.max(outputs, 1)

#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         # zero the parameter gradients
#                         optimizer.zero_grad()
#                         loss.backward()
#                         optimizer.step()

#                 # statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             epoch_loss = running_loss / len(dataloaders[phase].dataset)
#             epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())
#             if phase == 'val':
#                 val_acc_history.append(epoch_acc)
#             if phase == 'train':
#                 training_loss.append(epoch_acc)

#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model, training_loss, val_acc_history


def train_model(model, dataloaders, optimizer, num_epochs=25, device=DEVICE, loss_criteria = LOSS_CRITERIA):
    since = time.time()

    val_acc_history = []
    training_loss = []
    

    # best_model_wts = copy.deepcopy(model.state_dict())
    # best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    logits_h = model(inputs).logits.type(torch.cuda.FloatTensor)
                    labels = labels.type(torch.cuda.FloatTensor)
                    loss = loss_criteria(logits_h.view(-1), labels.view(-1))
                    # loss = loss_criteria(outputs, labels)

                    _, preds = torch.max(logits_h, 1)
                    _, real_labels = torch.max(labels, 1)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        # zero the parameter gradients
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == real_labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            # if phase == 'val' and epoch_acc > best_acc:
            #     best_acc = epoch_acc
            #     best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if phase == 'train':
                training_loss.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    # print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    # model.load_state_dict(best_model_wts)
    return model, training_loss, val_acc_history

In [3]:
train_dataset = AudioDataset(kind='train')
val_dataset = AudioDataset(kind='val')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

dataloaders = {
    'train':train_loader,
    'val':val_loader
}


In [4]:
# for inputs, labels in dataloaders['train']:
#     print(inputs)
#     print(labels)
#     break

In [5]:
# model, training_loss, val_acc_history = train_model(bert_model, dataloaders, optimizer, num_epochs=2, device=DEVICE, loss_criteria = LOSS_CRITERIA)


In [11]:
import numpy as np
training_loss = [i.cpu().numpy().tolist() for i in training_loss]
val_acc_history = [i.cpu().numpy().tolist() for i in val_acc_history]

NameError: name 'training_loss' is not defined

In [4]:

#ADAM: 
# Params: 
#     lr: 0.0012161288828451268
# SGD
# lr: 0.0004399074848905889
#     sgd_momentum: 0.782315314217227

# Params Adan: 
#     lr: 0.0020022613730624438
#     beta1: 0.046914599109421654
#     beta2: 0.019381017240822685
#     beta3: 0.010655285691959324



results_metric = {
    'Adam':{
        
    },
    'SGD':{
        
    },
    'Adan':{
        
    }
}

config = HubertConfig(
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=4,
    intermediate_size=256,
    num_labels=50,
    conv_dim=(512, 512, 512),
    conv_stride=(5, 2, 2),
    conv_kernel=(10, 3, 3)
    )
hubert_model = HubertForSequenceClassification(config).to(DEVICE)

final_optimizer_configurations = {
    'Adam': optim.Adam(hubert_model.parameters(), lr=1e-3),
    'SGD': torch.optim.SGD(hubert_model.parameters(), lr=4e-4, momentum=0.8, nesterov=True),
    'Adan': Adan(hubert_model.parameters(),lr = 1e-3, betas = (0.05, 0.02, 0.01), weight_decay = 0.02),
    # 'Madgrad': ....  
}

for optimizer_name in final_optimizer_configurations.keys():
    
    model, training_loss, val_acc_history = train_model(hubert_model, dataloaders, final_optimizer_configurations[optimizer_name], num_epochs=20, device=DEVICE, loss_criteria = LOSS_CRITERIA)
    results_metric[optimizer_name]['val_accuracy'] = val_acc_history
    results_metric[optimizer_name]['training_loss'] = training_loss
    results_metric[optimizer_name]['model'] = model
    # hubert_model = HubertForSequenceClassification(config).to(DEVICE)
# optim.Adam(model.parameters(), lr=3e-4)
# torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
# optim = Adan(model.parameters(),lr = 1e-3,                  # learning rate (can be much higher than Adam, up to 5-10x)
#     betas = (0.02, 0.08, 0.01), # beta 1-2-3 as described in paper - author says most sensitive to beta3 tuning
#     weight_decay = 0.02         # weight decay 0.02 is optimal per author
# )

Epoch 0/19
----------


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.81 GiB (GPU 0; 44.34 GiB total capacity; 38.30 GiB already allocated; 1.51 GiB free; 40.50 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
%matplotlib inline
def plot_results(train_losses,val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()